In [3]:
from escher import Builder
from IPython.display import display
import cobra
import pandas as pd
from cobra.io import read_sbml_model, write_sbml_model, save_json_model, load_json_model
import json

In [ ]:
model = cobra.test.read_sbml_model('/Users/hellpark/Desktop/Bioinformatics/model_objects/HalomonasBaseModel.sbml')
write_sbml_model(model, '/Users/hellpark/Desktop/Bioinformatics/model_objects/HalomonasCOBRAModel.sbml')
save_json_model(model, '/Users/hellpark/Desktop/Bioinformatics/model_objects/HalomonasCOBRAModel.json')

In [ ]:
# update the reaction names with the BIGG name
df = pd.ExcelFile('/Users/hellpark/Desktop/Bioinformatics/model_objects/HalomonasBaseModel.xls').parse('ModelReactions')
df.head()

In [ ]:
dfcomp = pd.ExcelFile('/Users/hellpark/Desktop/Bioinformatics/model_objects/HalomonasBaseModel.xls').parse('ModelCompounds')
dfcomp.head()    

In [ ]:
#replace all compounds with the bigg representation
for i in dfcomp.index:
    if dfcomp.at[i, 'BIGG'] is not None:
        oldcompoundid = dfcomp.at[i, 'id']
        for met in model.metabolites:
            if met.id == oldcompoundid:
                if str(dfcomp.at[i, 'BIGG']) != 'nan':
                    met.id = str(dfcomp.at[i, 'BIGG'])

In [ ]:
#replace all the reactions with BIGG representation
for i in df.index:
    if df.at[i,'bigg id'] is not None:
        oldrxnid = df.at[i,'id']
        for reaction in model.reactions:
            if reaction.id == oldrxnid:
                if str(df.at[i,'bigg id']) != 'nan':
                    #print(df.at[i,'bigg id'])
                    reaction.id = str(df.at[i,'bigg id'])

In [ ]:
#### below converts json to a cvs from halomonas annotated from kbase

In [4]:

    
with open('/Users/hellpark/Downloads/HalomonasAnnotated/4.json','r') as f:
    data = json.loads(f.read())

In [ ]:
multiple_level_data = pd.json_normalize(data, record_path =['cdss'], meta =['assembly_ref'], meta_prefix='config_params_', record_prefix='dbscan_')
multiple_level_data.to_csv('/Users/hellpark/Downloads/HalomonasAnnotated/multiplelevel_normalized_data.csv', index=False)

In [8]:

#chalmers model
with open('/Users/hellpark/Desktop/Bioinformatics/model_objects/Chalmers_halov1.json','r') as f:
    data = json.loads(f.read())
    
    
def parse_nested_json(json_d):
    result = {}
    for key in json_d.keys():
        if not isinstance(json_d[key], dict):
            result[key] = json_d[key]
        else:
            result.update(parse_nested_json(json_d[key]))
    return result
    
    
    
    
json_data = pd.json_normalize(data)
json_list = [j[1][0] for j in json_data.iterrows()]
parsed_list = [parse_nested_json(j) for j in json_list]
result = pd.DataFrame(parsed_list)
result.to_csv("/Users/hellpark/Desktop/Bioinformatics/model_objects/my_csv_file.csv", index=False)

AttributeError: 'list' object has no attribute 'keys'

In [12]:
multiple_level_data = pd.json_normalize(data, meta =['metabolites'], meta_prefix='config_params_', record_prefix='dbscan_')

normalized_df.to_csv('/Users/hellpark/Desktop/Bioinformatics/model_objects/my_csv_file2.csv',index=False)

In [ ]:
#### below reads in two excel pages of halomonas and finds the amino acid sequences in order to compare the CDS between models

In [ ]:
dfkbase = pd.ExcelFile('/Users/hellpark/Desktop/Bioinformatics/model_objects/HalomonasBaseModel.xlsm').parse('Halomonas WT TD01 Annotations')
dfkbase.head()

In [ ]:
dfchalm = pd.ExcelFile('/Users/hellpark/Desktop/Bioinformatics/model_objects/HalomonasBaseModel.xlsm').parse('OMICs')
dfchalm.head()

In [ ]:
# make a new column in halomonas kbase with the td1.0 data
for i in dfchalm.index:
    #td = dfchalm.at[i,'Fasta Number']
    tdn = dfchalm.at[i,'AA with *']
    for j in dfkbase.index:
        #ha = dfkbase.at[j,'Halomonas CDS']
        han = dfkbase.at[j,'Annotated translation']
        if str(han) in str(tdn):
            td = dfchalm.at[i,'Halomonas WT ann name'] = dfkbase.at[j,'Halomonas CDS']    

In [ ]:
for i in dfchalm.index:
    #td = dfchalm.at[i,'Fasta Number']
    tdn = dfchalm.at[i,'AA with *']
    for j in dfkbase.index:
        #ha = dfkbase.at[j,'Halomonas CDS']
        han = dfkbase.at[j,'Annotated translation']
        if str(tdn) in str(han):
            td = dfchalm.at[i,'Halomonas WT ann name'] = dfkbase.at[j,'Halomonas CDS']
dfchalm.head()

In [ ]:
#save again as text file
dfchalm.to_csv(path_or_buf='/Users/hellpark/Desktop/Bioinformatics/model_objects/np.csv')  

In [ ]:
# below takes in and takes the difference in proteomics data, then ranks / sorta based on biggest change

In [ ]:
ok = pd.read_excel(r'/Users/hellpark/Desktop/Bioinformatics/model_objects/HalomonasBaseModel.xlsm', 
                   sheet_name = 'OMICs')

In [ ]:
ok = pd.DataFrame(ok, columns=['Fasta Number','Halomonas WT ann name',
                                'Description in brief','NaCl  60g/L Protein pgDW', 'NaCl  20g/L Protein pgDW',
                                 'NaCl  100g/L Protein pgDW','Urea 3.6g/L Protein pgDW','Fermentation 9h Protein pgDW',
                                   'Fermentation 19h Protein pgDW','Fermentation 30h Protein pgDW','NaCl  60g/L RPKM',
                                    'NaCl  20g/L RPKM','NaCl  100g/L RPKM','Urea 3.6g/L RPKM','Fermentation 9h RPKM','Fermentation 19h RPKM',
                                    'Fermentation 30h RPKM', 'fold change'])

In [ ]:
ok[104:107]
ok['fold change'] = (ok['NaCl  20g/L Protein pgDW']-ok['NaCl  60g/L Protein pgDW'])/ok['NaCl  60g/L Protein pgDW']

In [ ]:
ok = ok.sort_values('fold change')

In [ ]:
# make a venn diagram
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
venn2(subsets = (22, 125,3833), set_labels = ('left=Halo Base / ', 'right=Chalmers GEM'))
plt.show()

In [ ]:
venn2(subsets = (22, 5, 2236), set_labels = ('left=Halo Base / ', 'right=Chalmers GEM'))
plt.show()

In [ ]:
# load an escher map
!curl --silent file:///Users/hellpark/Desktop/Bioinformatics/model_objects/HalomonasCOBRAModel.json > HalomonasCOBRAModel.json
model = cobra.io.load_json_model('HalomonasCOBRAModel.json')
difference_map = Builder(map_name='e_coli_core.Core metabolism')

In [ ]:
#add the model i'm working on, and highlight missing parts
difference_map.model_json = 'HalomonasCOBRAModel.json'
difference_map.highlight_missing = True